# This notebook is used to extract fixations for each participant

In [ ]:
from deepeye_analysis_package.getFixations import batch_extract_fixations

# Determine the environment ('Home' or 'Office') and set the data path accordingly
WHERE = 'Home'  # 'Office' or 'Home'

if WHERE == 'Home':
    path_to_data = r'C:/Users/aby600/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data'
else:
    # path = r'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'
    path_to_data =  r'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/withDelay/Approved/data'

batch_extract_fixations(path_to_data)